In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
import numpy as np

np.random.seed(42)

# Set your sample size
total_sampled_points = 5_000_000

# Automatically decide whether to sample with or without replacement
use_replacement = total_sampled_points > total_points

# Sample
all_indices = np.random.choice(total_points, size=total_sampled_points, replace=use_replacement)
dataset = dataset[all_indices]


# Optionally save the indices for reproducibility
# np.save("sampled_indices.npy", all_indices)


In [31]:
# --- Constants ---
SB = 10_000     # Small Batch
MB = 100_000    # Medium Batch
#LB = 1_000_000  # Large Batch
LB = total_sampled_points

SQ = 20         # Small Query
MQ = 100        # Medium Query
LQ = 500        # Large Query

# --- Generate test combinations ---
test_combinations = [(LB, SQ)]

In [38]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Sampled Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk from the *sampled* dataset
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [9]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------VP Tree---------------------**

In [10]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

**------------VP Tree Hybrid Opt-----------**

**------------------Run 1---------------------**

In [39]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

insert_time = 0.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 9.1749 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 6922.44it/s]

Total Search Time: 0.005188 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 9.1749 sec, Total Search Time: 0.005188 sec, Total Time: 9.1801 sec

Total Accumulated Time (All Batches): 9.1801 sec


**---------------Hybrid Opt Run 2------------------**

In [33]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

insert_time = 0.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 9.2514 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 7420.92it/s]

Total Search Time: 0.005455 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 9.2514 sec, Total Search Time: 0.005455 sec, Total Time: 9.2568 sec

Total Accumulated Time (All Batches): 9.2568 sec


**---------Hybrid Opt Run 3---------------------**

In [36]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 9.3176 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 5846.53it/s]

Total Search Time: 0.005931 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 9.3176 sec, Total Search Time: 0.005931 sec, Total Time: 9.3235 sec

Total Accumulated Time (All Batches): 9.3235 sec


**------Hybrid Option Run 4---------**

In [28]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 8.8634 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 6436.93it/s]

Total Search Time: 0.005503 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 8.8634 sec, Total Search Time: 0.005503 sec, Total Time: 8.8689 sec

Total Accumulated Time (All Batches): 8.8689 sec


**------Hybrid Avg and STD-----------**

In [40]:
import re
import numpy as np


results = [
    'Build Time: 9.1749 sec, Total Search Time: 0.005188 sec, Total Time: 9.1801 sec',
    'Build Time: 9.2514 sec, Total Search Time: 0.005455 sec, Total Time: 9.2568 sec',
    'Build Time: 9.3176 sec, Total Search Time: 0.005931 sec, Total Time: 9.3235 sec',
    'Build Time: 8.8634 sec, Total Search Time: 0.005503 sec, Total Time: 8.8689 sec'
]

# Regular expressions to extract the times
build_times = []
search_times = []
total_times = []

for line in results:
    build_match = re.search(r'Build Time:\s*([\d.]+)', line)
    search_match = re.search(r'Total Search Time:\s*([\d.]+)', line)
    total_match = re.search(r'Total Time:\s*([\d.]+)', line)

    if build_match and search_match and total_match:
        build_times.append(float(build_match.group(1)))
        search_times.append(float(search_match.group(1)))
        total_times.append(float(total_match.group(1)))

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Build Time", build_times)
stats("Search Time", search_times)
stats("Total Time", total_times)


Build Time - Mean: 9.1518 sec, Std Dev: 0.2009 sec, CV: 2.20%
Search Time - Mean: 0.0055 sec, Std Dev: 0.0003 sec, CV: 5.57%
Total Time - Mean: 9.1573 sec, Std Dev: 0.2010 sec, CV: 2.20%


**---------Original VP Tree----------------**

**---------Run 1----------------**

In [13]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 1
epsilon_rebuild = 0.0

insert_time = 0.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 390.7448 sec


Querying batch 1: 100%|██████████| 20/20 [00:05<00:00,  3.63it/s]

Total Search Time: 14.278013 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 390.7448 sec, Total Search Time: 14.278013 sec, Total Time: 405.0228 sec

Total Accumulated Time (All Batches): 405.0228 sec


**----------------Run 2---------------**

In [16]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 1
epsilon_rebuild = 0.0

insert_time = 0.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 372.5125 sec


Querying batch 1: 100%|██████████| 20/20 [00:05<00:00,  3.43it/s]

Total Search Time: 13.985321 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 372.5125 sec, Total Search Time: 13.985321 sec, Total Time: 386.4978 sec

Total Accumulated Time (All Batches): 386.4978 sec


**--------------Run 3--------------**

In [22]:
import time
from tqdm import tqdm
import numpy as np

# === Fixed Parameters ===
query_size = SQ
num_repetitions = query_size

leaf_size_rebuild = 1
epsilon_rebuild = 0.0

insert_time = 0.0

# Bookkeeping
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = dataset[start_idx:end_idx]

    # Accumulate data
    if len(cumulative_data):
        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
    else:
        cumulative_data = new_batch

    # Rebuild the VP Tree from scratch
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(cumulative_data)
    build_time = time.perf_counter() - build_start

    print(f"Rebuilt VP Tree with {len(cumulative_data):,} points.")
    print(f"Build Time: {build_time:.4f} sec")

    # Run queries
    queries = queries_list[batch_idx][:query_size]

    search_start = time.perf_counter()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Accumulated Time (All Batches): {grand_total_time:.4f} sec")


Batch 1: Adding 5,000,000 points
Rebuilt VP Tree with 5,000,000 points.
Build Time: 372.8534 sec


Querying batch 1: 100%|██████████| 20/20 [00:13<00:00,  1.48it/s]

Total Search Time: 13.514344 sec

=== Summary of VP Tree Scaling Test ===
Points: 5,000,000, Build Time: 372.8534 sec, Total Search Time: 13.514344 sec, Total Time: 386.3677 sec

Total Accumulated Time (All Batches): 386.3677 sec


**---------Original VP Tree Avg and Std------**

In [23]:
import re
import numpy as np


results = [
    'Build Time: 390.7448 sec, Total Search Time: 14.278013 sec, Total Time: 405.0228 sec',
    'Build Time: 372.5125 sec, Total Search Time: 13.985321 sec, Total Time: 386.4978 sec',
    'Build Time: 372.8534 sec, Total Search Time: 13.514344 sec, Total Time: 386.3677 sec'
]

# Regular expressions to extract the times
build_times = []
search_times = []
total_times = []

for line in results:
    build_match = re.search(r'Build Time:\s*([\d.]+)', line)
    search_match = re.search(r'Total Search Time:\s*([\d.]+)', line)
    total_match = re.search(r'Total Time:\s*([\d.]+)', line)

    if build_match and search_match and total_match:
        build_times.append(float(build_match.group(1)))
        search_times.append(float(search_match.group(1)))
        total_times.append(float(total_match.group(1)))

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Build Time", build_times)
stats("Search Time", search_times)
stats("Total Time", total_times)


Build Time - Mean: 378.7036 sec, Std Dev: 10.4294 sec, CV: 2.75%
Search Time - Mean: 13.9259 sec, Std Dev: 0.3853 sec, CV: 2.77%
Total Time - Mean: 392.6294 sec, Std Dev: 10.7332 sec, CV: 2.73%
